In [1]:
# Using webscraping env
import datetime, os
import re, time, requests
import pandas as pd
from parsel import Selector
from selenium import webdriver
from selenium.webdriver import *
from selenium.webdriver.chrome.options import Options

## Scrapting data

In [2]:
def get_driver(url):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(
        executable_path="/home/danph/Repos/run/drivers/chromedriver",
        options=chrome_options)
    driver.get(url)
    return driver

In [3]:
url = "https://www.coronavirus.vic.gov.au/exposure-sites"
driver = get_driver(url)
all_sites = [] # Store info of all sites

### Click to different pages

In [4]:
def get_site_info(site):
    """This function get info for each site"""
    site_name = site.xpath("td[2]/div/span[1]/text()").get()
    address = site.xpath("td[2]/div/span[2]/text()").get()
    suburb = site.xpath("td[2]/div/span[3]/text()").get()
    postcode = site.xpath("td[2]/div/span[5]/text()").get()
    period = site.xpath("td[3]/span[2]/text()").get()
    note = site.xpath("td[4]/span[2]/text()").get()
    date_added = site.xpath("td[5]/span[2]/text()").get()
    advice = site.xpath("td[6]/div/button/span/text()").get()
    return (site_name, address, suburb, postcode, period, note, date_added, advice)

def get_sites(driver):
    """This function get sites in one page"""
    sites = Selector(driver.page_source)
    sites = sites.xpath('//div/table[@class="rpl-search-results-table"]/tbody/tr')
    
    for site in sites:
        all_sites.append(get_site_info(site))

In [ ]:
%%time
# Get data in all pages
## Click each page and get data in each table
while True:
    try:
        get_sites(driver)
        time.sleep(3)
        next_page = driver.find_element_by_xpath("//nav[@class='rpl-pagination']/div/button[last()]")
        next_page.click()        
    except Exception:
        print("Finished")
        break

### Save into a dataframe

In [ ]:
sites_df = pd.DataFrame(all_sites, columns=['site_name','address','suburb', 'postcode', 'period', 'note', 'date_added','advice'])
sites_df.shape

In [ ]:
sites_df.head()

In [8]:
sites_df.isna().sum()

site_name     0
address       0
suburb        0
postcode      0
period        0
note          0
date_added    0
advice        0
dtype: int64

### Basic data processing and save into a csv file

In [9]:
def get_tier(text):
    if text.find("Tier 1") > -1:
        res = "Tier 1"
    elif text.find("Tier 2") > -1:
        res = "Tier 2"
    else:
        res = "Tier 3"
    return res

sites_df['tier'] = sites_df.advice.apply(lambda x: get_tier(x))

In [10]:
# TODO: Clean address
sites_df['full_address'] = sites_df['address'] + ", " + sites_df['suburb'] + ", Victoria, " + sites_df['postcode'].astype(str) + ", Australia"
# Save into json file
sites_df.to_json("../data/covid_sites.json")